In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
air_humidity = pd.read_csv('../data/formatted/air_humidityformatted_data.csv')
pressure = pd.read_csv('../data/formatted/pressureformatted_data.csv')
rainfall = pd.read_csv('../data/formatted/rainfallformatted_data.csv')

In [3]:
print(air_humidity.head())

         date    department  avg_value
0  01/01/2005     ANTIOQUIA  76.600000
1  01/01/2005         CAUCA  76.636364
2  01/01/2005  CUNDINAMARCA  63.800000
3  01/01/2006     ANTIOQUIA  72.291080
4  01/01/2006         CAUCA  85.405488


In [4]:
print(pressure.head())

         date    department   avg_value
0  01/01/2005     ANTIOQUIA  752.281169
1  01/01/2005         CAUCA  744.227273
2  01/01/2006     ANTIOQUIA  804.622857
3  01/01/2006         CAUCA  890.585211
4  01/01/2006  CUNDINAMARCA  824.049576


In [5]:
print(rainfall.head())

         date    department  total_value  avg_value
0  01/01/2004  CUNDINAMARCA          0.0   0.000000
1  01/01/2005     ANTIOQUIA          0.0   0.000000
2  01/01/2005  CUNDINAMARCA        158.5   0.600379
3  01/01/2006     ANTIOQUIA          0.0   0.000000
4  01/01/2006         CAUCA          0.0   0.000000


In [11]:
# Convertir la columna 'date' a formato datetime en cada DataFrame
air_humidity['date'] = pd.to_datetime(air_humidity['date'], format='%d/%m/%Y')
pressure['date'] = pd.to_datetime(pressure['date'], format='%d/%m/%Y')
rainfall['date'] = pd.to_datetime(rainfall['date'], format='%d/%m/%Y')

# Extraer año, mes y día en cada DataFrame
air_humidity['year'] = air_humidity['date'].dt.year
air_humidity['month'] = air_humidity['date'].dt.month
air_humidity['day'] = air_humidity['date'].dt.day

pressure['year'] = pressure['date'].dt.year
pressure['month'] = pressure['date'].dt.month
pressure['day'] = pressure['date'].dt.day

rainfall['year'] = rainfall['date'].dt.year
rainfall['month'] = rainfall['date'].dt.month
rainfall['day'] = rainfall['date'].dt.day

# Unir los DataFrames en uno solo
df = air_humidity.merge(pressure, on=['year', 'month', 'day', 'department'], suffixes=('_air_humidity', '_pressure'))
df = df.merge(rainfall, on=['year', 'month', 'day', 'department'], suffixes=('', '_rainfall'))

# Renombrar columnas
df.rename(columns={'avg_value': 'avg_value_rainfall', 'total_value': 'total_value_rainfall'}, inplace=True)

# Consolidar datos para evitar duplicados antes de pivotar
df_grouped = df.groupby(['year', 'department', 'day'], as_index=False).mean()

# Crear las columnas de los primeros 12 días del año
final_df = df_grouped.pivot(index=['year', 'department'], columns='day', values=['avg_value_air_humidity', 'avg_value_pressure', 'avg_value_rainfall', 'total_value_rainfall'])

# Filtrar solo los primeros 12 días
final_df = final_df.loc[:, (slice(None), range(1, 13))]

# Renombrar columnas para incluir 'day_X'
final_df.columns = [f"day_{col[1]}_{col[0]}" for col in final_df.columns]

# Calcular los promedios mensuales
monthly_avg = df.groupby(['year', 'month', 'department'])[['avg_value_air_humidity', 'avg_value_pressure', 'avg_value_rainfall', 'total_value_rainfall']].mean()

# Pivotear para obtener columnas de meses
monthly_avg = monthly_avg.pivot_table(index=['year', 'department'], columns='month')

# Renombrar columnas para incluir 'month_X'
monthly_avg.columns = [f"month_{col[1]}_{col[0]}" for col in monthly_avg.columns]

# Unir ambos DataFrames
final_df = final_df.reset_index().merge(monthly_avg.reset_index(), on=['year', 'department'])


In [16]:
final_df.to_csv('../data/final/merged_data.csv', index=False)

In [17]:
final_df

,year,department,day_1_avg_value_air_humidity,day_2_avg_value_air_humidity,day_3_avg_value_air_humidity,day_4_avg_value_air_humidity,day_5_avg_value_air_humidity,day_6_avg_value_air_humidity,day_7_avg_value_air_humidity,day_8_avg_value_air_humidity,...,month_3_total_value_rainfall,month_4_total_value_rainfall,month_5_total_value_rainfall,month_6_total_value_rainfall,month_7_total_value_rainfall,month_8_total_value_rainfall,month_9_total_value_rainfall,month_10_total_value_rainfall,month_11_total_value_rainfall,month_12_total_value_rainfall
0,2005,ANTIOQUIA,79.852941,80.672811,81.346570,82.119394,81.964511,80.682940,80.415075,79.577744,...,NaN,NaN,9.031250,14.193333,7.974194,6.774194,9.710000,29.690323,15.780000,13.151613
1,2005,CAUCA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.630769,7.293333
2,2005,CUNDINAMARCA,80.521367,80.860739,81.684535,81.360867,79.129287,81.438005,80.040793,78.780543,...,9.164516,38.150,63.170968,28.950000,28.338710,35.193548,35.916667,55.712903,42.373333,24.364516
3,2005,NORTE DE SANTANDER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.430000,NaN,NaN,0.025000
4,2006,ANTIOQUIA,78.326688,77.824422,79.471554,79.966208,79.404422,79.364075,81.427893,82.115399,...,11.061290,10.805,11.225806,16.580000,3.432258,5.519355,5.530000,2.183871,3.966667,5.186667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,2025,CUNDINAMARCA,82.666875,85.997453,94.665964,91.645608,95.906185,89.885676,88.135713,85.524157,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138,2025,GUAINÍA,98.344444,95.283333,85.620513,89.969048,86.670238,84.439927,82.702564,87.476786,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
139,2025,NARIÑO,40.827302,37.593002,38.703550,39.743658,37.762539,36.705635,36.636399,36.585440,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
140,2025,NORTE DE SANTANDER,85.238506,83.953458,85.100738,87.641485,91.205914,86.274151,75.121906,81.089590,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
